In [10]:
import numpy as np
import matplotlib.pyplot as plt
import batman

# try  
#   pip install lmfit
# if you don't have it installed
from lmfit import minimize,Parameters,Parameter,report_fit

In [ ]:
def transitmin(params, time, flux, fluxerr):
    """ Calculate the transit light curve and the difference from the observed flux """
    parbat = batman.TransitParams()
    parbat.t0 = params['t0'].value      # time of transit
    parbat.per = params['per'].value    # orbital period
    parbat.rp = params['rp'].value      # planet radius (in units of stellar radii)
    parbat.a = params['a'].value        # semi-major axis (in units of stellar radii)
    parbat.inc = params['inc'].value    # orbital inclination (in degrees)
    parbat.ecc = params['ecc'].value    # eccentricity
    parbat.w = params['w'].value        # longitude of periastron (in degrees)
    parbat.u = [params['u1'].value, params['u2'].value]    # limb darkening coefficients
    parbat.limb_dark = "quadratic"                         # limb darkening model

    m = batman.TransitModel(parbat, time, supersample_factor=21, exp_time=0.5/24.)
    model = m.light_curve(parbat)
    return (model - flux)/fluxerr

In [ ]:
## read the light curve
filename = 'ktwo211089792.txt'

time, lc, lcerror = 


## plot the light curve
# ...
plt.xlabel('Time [days]')
plt.ylabel('Flux')

In [ ]:
# parameters obtained from the BLS
period = 
epoch = 
depth = 
duration = 


time = time-epoch

# set up iniital parameters
# estimate a/rs and rp/rs
g = 6.67428e-11
gm = 1.32712440041e20
msun = gm/g
rsun = 6.95508e8
sdensity = 1.0
aoverr = (sdensity*(gm*(period*24.*3600.0)**2.)/(4.*np.pi**2.*rsun**3.))**(1./3.)
rp = np.sqrt(depth)

# Assume  limb darkening
u1 = 0.4983
u2 = 0.2042

In [ ]:
# set up the parameters for batman
params2 = batman.TransitParams()
params2.t0 = 0.0                       #time of inferior conjunction
params2.per = period                    #orbital period
params2.rp = rp                     #planet radius (in units of stellar radii)
params2.a = aoverr                       #semi-major axis (in units of stellar radii)
params2.inc = 89.                     #orbital inclination (in degrees)
params2.ecc = 0.                      #eccentricity
params2.w = 90.                       #longitude of periastron (in degrees)
params2.u = [u1, u2]                #limb darkening coefficients
params2.limb_dark = "quadratic"       #limb darkening model



# calculate and plot the starting model
m = batman.TransitModel(params2, time, supersample_factor = 21, exp_time = 0.5/24.)    #initializes model
model = m.light_curve(params2)

plt.figure()
plt.plot(time, lc, '.')
plt.plot(time,  model, color='r')
plt.xlabel("Time")
plt.ylabel("Relative flux")
plt.show()

In [ ]:
# create a set of parameters for lmfit
# the ones that are fixed have vary=false
params = Parameters()
params.add('t0', value = 0.0)
params.add('per', value = period, vary = True)
params.add('rp', value = rp)
params.add('a', value = aoverr)
params.add('inc', value = 88.)
params.add('ecc', value = 0.0, vary = False)
params.add('w', value = 90., vary = False)
params.add('u1', value = u1, vary = False)
params.add('u2', value = u2, vary = False)

In [8]:
# use lmfit to estimate the transit parameters
result = minimize(transitmin,params, args=(time, lc, lcerror))

report_fit(result)

NameError: name 'params' is not defined

In [ ]:
# recover the fitted parameters from the result
parbat = batman.TransitParams()
parbat.t0 = result.params['t0'].value                     #time of inferior conjunction
parbat.per = result.params['per'].value                    #orbital period
parbat.rp = result.params['rp'].value                      #planet radius (in units of stellar radii)
parbat.a = result.params['a'].value                      #semi-major axis (in units of stellar radii)
parbat.inc = result.params['inc'].value                     #orbital inclination (in degrees)
parbat.ecc = 0.                      #eccentricity
parbat.w = 90.                       #longitude of periastron (in degrees)
parbat.u = [u1, u2]                #limb darkening coefficients
parbat.limb_dark = "quadratic"       #limb darkening model

period = result.params['per'].value
epoch0 = result.params['t0'].value

#phase fold to plot
phase0 = (time )/period
phase = phase0%1
phadur = duration*2.2/period

phase[np.where(phase>0.5)[0]]-=1
cond = np.where(abs(phase) < phadur)[0]

timefold = phase[cond]*period
fluxfold = lc[cond]

fluxfold = fluxfold[np.argsort(timefold)]
timefold = np.sort(timefold)


# calculate and plot the two models initial model for the time folded dataset
m = batman.TransitModel(params2, timefold, supersample_factor = 21, exp_time = 0.5/24.)
model = m.light_curve(params2)


m = batman.TransitModel(parbat, timefold, supersample_factor = 21, exp_time = 0.5/24.)
fitmodel = m.light_curve(parbat)






In [9]:
plt.figure()
plt.plot(timefold, fluxfold, '.')
plt.plot(timefold,  model, color='g')
plt.plot(timefold,  fitmodel, color='r')
plt.xlabel("Time")
plt.ylabel("Relative flux")
plt.show()

NameError: name 'timefold' is not defined

In [ ]:
#### Cut the transits and write them to a file
filename = 'cuttransits.txt'

fp = open(filename, 'w')
for i in cond:
    fp.write('%.10f\t%f\t%f\n'%(time[i]+epoch, lc[i], lcerror[i]))
fp.close()